In [ ]:
from retrievalAugmentedTransformer import RAT, Data_Processing

In [ ]:
dp = Data_Processing()
df = dp.get_data("AAPL", "2020-01-01", "2025-04-02")

In [ ]:
print(df.shape)
df = dp.load_features("AAPL", df)
print(df.shape)

In [ ]:
rat = RAT("AAPL", len(df.columns))
X, y = dp.format_data_combined(df, rat, 10, 5)

In [ ]:
rat.query_articles("AAPL")

In [ ]:
rat.load_model()

In [ ]:
scale_preds, preds = rat.predict(X[-20:])

In [ ]:
print(X[-20:].shape)
scale_preds.shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(X[-20:, 0, :2], color = "red")
plt.plot(scale_preds[:, 0], color = "cyan")

In [ ]:
rat.save_model()